In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import numpy as np
from dipy.sims.voxel import multi_tensor, multi_tensor_odf
from dipy.data import get_data, get_sphere
from dipy.core.gradients import gradient_table
from dipy.reconst.dsi import DiffusionSpectrumModel

import json
from diGP.preprocessing import get_HCP_loader
from diGP.dataManipulations import DataHandler
from diGP.model import Model, get_default_independent_kernel
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
with open('../config.json', 'r') as json_file:
    conf = json.load(json_file)

subject_path = conf['HCP']['data_paths']['mgh_1007']

In [ ]:
gtab = {}
data = {}
odf = {}

In [ ]:
loader = get_HCP_loader(subject_path)

gtab['train'] = loader.gtab
gtab['train'].info

For the testing we will use a standard DSI acqusition scheme with 514 gradient directions and 1 S0.

In [ ]:
btable = np.loadtxt(get_data('dsi515btable'))

gtab['test'] = gradient_table(btable[:, 0], btable[:, 1:],
                              big_delta=gtab['train'].big_delta, small_delta=gtab['train'].small_delta)
gtab['test'].info

Let’s create a multi tensor with 2 fiber directions at 60 degrees.

In [ ]:
evals = np.array([[0.0015, 0.0003, 0.0003],
                  [0.0015, 0.0003, 0.0003]])

directions = [(-30, 0), (30, 0)]

fractions = [50, 50]

for key, _gtab in gtab.items():
    data[key] = multi_tensor(_gtab, evals, 100, angles=directions,
                             fractions=fractions, snr=None)[0][None, None, None, :]

In [ ]:
sphere = get_sphere('symmetric724').subdivide(1)

odf['gt'] = multi_tensor_odf(sphere.vertices, evals, angles=directions,
                             fractions=fractions)[None, None, None, :]

Perform the reconstructions with standard DSI.

In [ ]:
dsi_model = DiffusionSpectrumModel(gtab['test'])
dsi_fit = dsi_model.fit(data['test'])

odf['dsi'] = dsi_fit.odf(sphere)

In [ ]:
#dsi_model.filter
#plt.plot(gtab['train'].qvals)
plt.plot(data['test'].flatten())
#plt.plot(data['pred'].flatten())
plt.plot(dsi_fit.data.flatten() * dsi_model.filter)

Predict the test data values using GP

In [ ]:
data_handler = DataHandler(gtab['train'], data=data['train'])
data_handler_pred = DataHandler(gtab['test'], data=None, spatial_shape=(1, 1, 1))
kernel = get_default_independent_kernel(data_handler)

gp_model = Model(data_handler, kernel, data_handler_pred=data_handler_pred, grid_dims=[[0], [1], [2], [3, 4, 5, 6]],
                 verbose=True)

gp_model.train()

data['pred'] = gp_model.predict()


Feed the estimate to the same dsi model fit as above.

In [ ]:
odf['gp'] = dsi_model.fit(data['pred']).odf(sphere)

Visualize the ground truth ODF together with the DSI.

In [ ]:
from dipy.viz import fvtk

ren = fvtk.ren()

odfs = np.vstack((odf['gt'], odf['dsi'], odf['gp']))

odf_actor = fvtk.sphere_funcs(odfs, sphere)
odf_actor.RotateX(90)
fvtk.add(ren, odf_actor)
fvtk.record(ren, out_path='dsi_sim.png', size=(300, 300))